In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [1]:
%run Partition.ipynb
%run myutil.ipynb

In [3]:
# 1개 라인이 가지는 객체 정보
class Line:
    PRINT_PARAM = False
    DEBUG_POINT = False
    DEBUG_LABEL = False

    def __init__(self, p_num, label, linewidth, threshold=(2.9,0.3)):
        self.partition_num = p_num
        self.threshold = threshold
        self.label = label
        self.linewidth = linewidth
        self.sample_num = random.randint(10, 50)
        self.g_list, self.g_list_num = self._growth_type(p_num)
        self.init_x, self.init_y = self._init_pass_point(p_num)
        
        self.p_list, self.x, self.y = self._points(p_num)
        
    def _monotonic(self):
        pnum = self.partition_num
        monotonic = 0
        sign_value = 0
        for p in self.p_list:
            sign_value = sign_value + p.sign()

        if sign_value == pnum:
            monotonic = 1
        elif sign_value == -pnum:
            monotonic = 2
            
        return monotonic
    
        
    def _points(self, pn):
        line_x = np.empty(0)
        line_y = np.empty(0)
        p_list = []
        for i in range(pn):
            px = self.init_x[i:i+2]
            py = self.init_y[i:i+2]
            
            partition = Partition(self.g_list[i], px, py, self.sample_num)
 
            params, _ = partition.params()
            if self.PRINT_PARAM is True:
                    print("[PARAMETER]\n{0}".format(params))
            
            points = partition.points(self.g_list[i], np.asarray(params))

            xfit = [x for x,y in points]
            yfit = [y for x,y in points]
            
            line_x = np.concatenate((line_x, xfit))
            line_y = np.concatenate((line_y, yfit))
            
            p_list.append(partition)

        return p_list, line_x, line_y
        
    def _init_pass_point(self, n):
        th1 = self.threshold[0]
        th2 = self.threshold[1]
        
        cont_flag = False
        num = 0
        while True:
            num = num + 1
            cont_flag = False

            x = np.random.uniform(0, 1, n+1)
            y = np.random.uniform(0, 1, n+1)
            x.sort()

            init_slope = []
            for j in range(n):
                init_slope.append((y[j+1]-y[j]) / (x[j+1]-x[j]))

            # 연속된 두 점이 일정 기울기 범위에 있지 않으면 다시 생성
            for m in init_slope:
                if np.abs(m) > th1 or np.abs(m) < th2:
                    cont_flag = True
                    break

            if cont_flag is False:
                break
                
            if num > 1000:
                break

        return (x,y)
    
    def _partition_number(self):
        return random.randint(1, 3)
    
    def _growth_type(self, p_num):
        r_list = []
        r_list_num = []
        for i in range(p_num):
            r = random.choice(['linear', 'log', 'exponential'])
            r_list.append(r)
            r_list_num.append(self.growthtype_to_num(r))
            
        return r_list, r_list_num
    
    def growthtype_to_num(self, r):
        idx = 0
        if r == 'linear':
            idx = 0
        elif r == 'log':
            idx = 2
        elif r == 'exponential':
            idx = 4
        
        return idx


    def draw(self, fmt=''):   
        if self.DEBUG_POINT is True:
            plt.plot(self.init_x, self.init_y, 'o')
        
        plt.plot(self.x, self.y, fmt, label=self.label, linewidth=self.linewidth)
        
        if self.DEBUG_LABEL is True:
            print("my label : ", self.label)
       